<a href="https://colab.research.google.com/github/pablohenrique93/projeto-de-etl-comunicacao-de-acidente-de-trabalho-cat/blob/main/projeto_etl_cat_ivana_pablo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### INSTALAÇÃO BIBLIOTECAS

In [ ]:
#INSTALAÇÃO DO PANDERA, PYSPARK E GOOGLE CLOUD RESPECTIVAMENTE.
!pip install PYSPARK
!pip install gcsfs

### IMPORTAÇÃO BIBLIOTECAS

In [ ]:
#IMPORTAÇÃO DAS BIBLIOTECAS 
from pyspark.sql import SparkSession
import pyspark.sql.functions
from pyspark.sql.functions import *
import os
import pyspark.sql.functions as F
import pandas as pd
import numpy as np
from pyspark.sql.window import Window
from google.cloud import storage
from pyspark.sql.types import *
import pymongo
from pymongo import MongoClient

# EXTRAÇÃO DA BASE DE DADOS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
serviceAccount = '/content/aulas-bc26-ivana-b8f6bd3e55b9.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [ ]:
# FAZER ACESSO AO BUCKET
client = storage.Client()
#client = storage.Client()
# CRIAR UMA VARIÁVEL PARA RECEBER O NOME DA BUCKET
bucket = client.get_bucket('bc26-datasets-ivana')
blob1 = bucket.blob('D.SDA.PDA.005.CAT.202201 - D.SDA.PDA.005.CAT.202201_orig.csv')
blob2 = bucket.blob('D.SDA.PDA.005.CAT.202204 - D.SDA.PDA.005.CAT.202204_orig.csv')
blob3 = bucket.blob('D.SDA.PDA.005.CAT.202210 - D.SDA.PDA.005.CAT.202210_orig.csv')
# CRIAR UMA VARIÁVEL PARA RECEBER O CAMINHO DO ARQUIVO
path1 ='gs://bc26-datasets-ivana/projeto aquecimento/D.SDA.PDA.005.CAT.202201 - D.SDA.PDA.005.CAT.202201_orig.csv'
path2 = 'gs://bc26-datasets-ivana/projeto aquecimento/D.SDA.PDA.005.CAT.202204 - D.SDA.PDA.005.CAT.202204_orig.csv'
path3 = 'gs://bc26-datasets-ivana/projeto aquecimento/D.SDA.PDA.005.CAT.202210 - D.SDA.PDA.005.CAT.202210_orig.csv'
df_pd_1 = pd.read_csv(path1, low_memory=False)
df_pd_2 = pd.read_csv(path2, low_memory=False)
df_pd_3 = pd.read_csv(path3, low_memory=False)

# PRÉ-ANALISE DOS DF'S ORIGINAIS

In [ ]:
#ANÁLISE DOS DF1, DF2 E DF3 RESPECTIVAMENTE
df_pd_1.head(100)

In [ ]:
#VERFICAÇÃO DA TIPAGEM DAS COLUNAS
df_pd_1.dtypes

In [ ]:
df_pd_2.dtypes

In [ ]:
df_pd_3.dtypes

In [ ]:
df_pd_1.shape

In [ ]:
df_pd_2.shape

In [ ]:
df_pd_3.shape

In [ ]:
df_pd_1.count

In [ ]:
df_pd_2.count

In [ ]:
df_pd_3.count

In [ ]:
df_pd_1.head(10)

In [ ]:
df_pd_2.head(10)

# VERIFICAÇÃO DE INCONSISTÊNCIAS NOS DF'S INDIVIDUALMENTE

In [ ]:
#POR MOTIVOS DE PROCESSAMENTO OPTAMOS POR VERIFICAR A COLUNA 'CNPJ/CEI Empregador' INDIVIDUALMENTE DE CADA DF.

'df_pd_1'

valores_estranhos = []
for i in range(len(df_pd_1)):
  try:
    int(df_pd_1.loc[i , 'CNPJ/CEI Empregador'])
  except Exception:
    valores_estranhos.append(df_pd_1.loc[i , 'CNPJ/CEI Empregador'])
print(valores_estranhos)


In [ ]:
'df_pd_2'

valores_estranhos = []
for i in range(len(df_pd_2)):
  try:
    int(df_pd_2.loc[i , 'CNPJ/CEI Empregador'])
  except Exception:
    valores_estranhos.append(df_pd_2.loc[i , 'CNPJ/CEI Empregador'])
print(valores_estranhos)

In [ ]:
'df_pd_3'

valores_estranhos = []
for i in range(len(df_pd_3)):
  try:
    int(df_pd_3.loc[i , 'CNPJ/CEI Empregador'])
  except Exception:
    valores_estranhos.append(df_pd_3.loc[i , 'CNPJ/CEI Empregador'])
print(valores_estranhos)

In [ ]:
df_pd_3.dtypes

In [ ]:
df_pd_3.dtypes

# CONCATENAÇÃO DOS DF'S ORIGINAIS

In [ ]:
# CONCATENAÇÃO DOS 3 DF'S ORIGINAIS PARA UM ÚNICO DF
df_pd_final = [df_pd_1, df_pd_2, df_pd_3]
df_pd_final = pd.concat(df_pd_final)

In [ ]:
df_pd_final_teste = df_pd_final.copy
df_pd_final_teste = df_pd_final

In [ ]:
df_pd_final_teste.dtypes

In [ ]:
# ENVIO DO DF CONCATENADO PARA A BUCKET NA GCP
df_pd_final.to_csv('gs://bc26-datasets-ivana/catweb_orig_export.csv',
               storage_options={'token':'/content/aulas-bc26-ivana-b8f6bd3e55b9.json'}
)

In [ ]:
#CONTAGEM DO TOTAL DE LINHAS NO DF
len(df_pd_final.index)

# BACKUP DOS DF'S

In [ ]:
#ANTES DE INICIAR O TRATAMENTO DE DADOS FAZ-SE NECESSÁRIO UM BACKUP, PARA EM CASO DE UM POSSÍVEL ERRO, OS DADOS ESTAREM ASSEGURADOS
bckup_df_pd_1 = df_pd_1.copy
bckup_df_pd_2 = df_pd_2.copy
bckup_df_pd_3 = df_pd_3.copy
bckup_df_pd_final = df_pd_final.copy

#ENVIO DOS DF'S PARA O MongoDB

In [ ]:
#CONECTOR DO MONGO ATLAS
uri = "uri_mongo_db"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='certificate_key')

In [ ]:
#ESCOLHA DO DATABASE E COLEÇÃO - ORIG
db = client['catweb_db_orig'] #DATABASE
colecao_catweb = db['catweb_orig'] #COLEÇÃO
db_tratado = client['catweb_db_tratado']
collection_tratado = db['catweb_tratado']

In [ ]:
#ENVIO DOS DF'S POR PARTES 1/2/3 RESPECTIVAMENTE
df_pd_1_dict = df_pd_1.to_dict("records")
colecao_catweb.insert_many(df_pd_1_dict)

In [ ]:
df_pd_2_dict = df_pd_2.to_dict("records")
colecao_catweb.insert_many(df_pd_2_dict)

In [ ]:
df_pd_3_dict = df_pd_3.to_dict("records")
colecao_catweb.insert_many(df_pd_3_dict)

In [ ]:
colecao_catweb.count_documents({})

247718

In [ ]:
#ENVIO DO DF CONCATENADO 
df_pd_final_dict = df_pd_final.to_dict("records")
colecao_catweb.insert_many(df_pd_final_dict)

# PRÉ-ANÁLISE DO DF CONCATENADO

In [ ]:
#PRÉ ANÁLISE COM O OBJETIVO DE ANALISAR O DF CONCATENADO/DISPOSIÇÃO DAS COLUNAS ASSIM COMO A PRÉVIA DE COMO ESTÃO A TIPAGEM DAS COLUNAS.

In [ ]:
df_pd_final.dtypes

In [ ]:
df_pd_final.shape

In [ ]:
df_pd_final.count

In [ ]:
df_pd_final.head(10)

In [ ]:
df_pd_final.tail(10)

In [ ]:
pd.set_option('display.max_columns',100)

In [ ]:
df_pd_3.tail(10)

In [ ]:
df_pd_2.tail(10)

In [ ]:
df_pd_1.tail(10)

In [ ]:
df_pd_3.head(10)

# TRATAMENTO DE DADOS

---



In [ ]:
#VERIFICAÇÃO DE CADA COLUNA COM O OBJETIVO DE ENCONTRAR POSSÍVEIS INCONSISTÊNCIAS.

In [ ]:
df_pd_final.head()

In [ ]:
pd.unique(df_pd_final['Agente  Causador  Acidente'])

In [ ]:
pd.unique(df_pd_final['Data Acidente'])

In [ ]:
pd.unique(df_pd_final['CBO'])

In [ ]:
pd.unique(df_pd_final['CID-10'])

In [ ]:
pd.unique(df_pd_final['CNAE2.0 Empregador'])

In [ ]:
pd.unique(df_pd_final['CNAE2.0 Empregador.1'])

In [ ]:
pd.unique(df_pd_final['Emitente CAT'])

In [ ]:
pd.unique(df_pd_final['Espécie do benefício'])

In [ ]:
pd.unique(df_pd_final['Filiação Segurado'])

In [ ]:
pd.unique(df_pd_final['Indica Óbito Acidente']) 

In [ ]:
pd.unique(df_pd_final['Munic Empr'])

In [ ]:
pd.unique(df_pd_final['Natureza da Lesão'])

In [ ]:
pd.unique(df_pd_final['Origem de Cadastramento CAT'])

In [ ]:
pd.unique(df_pd_final['Parte Corpo Atingida'])

In [ ]:
pd.unique(df_pd_final['Sexo'])

In [ ]:
pd.unique(df_pd_final['Tipo do Acidente'])

In [ ]:
pd.unique(df_pd_final['UF  Munic.  Acidente'])

In [ ]:
pd.unique(df_pd_final['UF Munic. Empregador'])

In [ ]:
pd.unique(df_pd_final['Data Acidente.1'])

In [ ]:
pd.unique(df_pd_final['Data Despacho Benefício'])

In [ ]:
pd.unique(df_pd_final['Data Acidente.2'])

In [ ]:
pd.unique(df_pd_final['Data Emissão CAT'])

In [ ]:
pd.unique(df_pd_final['Data Nascimento'])

In [ ]:
pd.unique(df_pd_final['CNPJ/CEI Empregador'])

#LIMPEZA DE INCONSISTÊNCIAS ENCONTRADAS

In [ ]:
#SUBSTITUIÇÃO DA INCONSISTÊNCIA '{ñ class}' POR 'n_classificado' ATRIBUINDO ASSIM O SEU SIGNIFICADO E REMOVENDO AS CHAVES PARA QUE NÃO 
#HAJA PROBLEMA EM FUTURAS MANIPULAÇÕES DOS DADOS, INICIALMENTE TAMBEM FOI NECESSÁRIO RENOMEAR ALGUMAS COLUNAS 
#PARA FACILITAR A MANIPULAÇÃO DOS DADOS

In [ ]:
df_pd_final_teste.rename(columns={'Agente  Causador  Acidente':'ag_causador_acidente'}, inplace=True)

In [ ]:
df_pd_final_teste.replace({'ag_causador_acidente':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.ag_causador_acidente

In [ ]:
#SUBSTITUINDO '{ñ class}' e '{ñ' por '000000-n_classificado', A INTENÇÃO DESTA SUBSTITUIÇÃO E INSERÇÃO DE UM CÓDIGO 'ZERADO' É PADRONIZAR A COLUNA JÁ QUE OS CBO'S POSSUEM 
#UMA ESPÉCIE DE CÓDIGO QUE O ANTECEDE, JÁ PLANEJANDO TAMBEM UMA POSSÍVEL SEPARAÇÃO DOS MESMOS, CRIANDO UMA NOVA COLUNA QUE RECEBERÁ O CÓDIGO. 
# bckup_df_pd_final.rename(columns={'CBO':'cbo'}, inplace=True)
df_pd_final_teste.replace({'CBO':'{ñ'}, '000000-n_classificado', regex=True, inplace=True)
df_pd_final_teste.replace({'CBO':'class}'}, '000000-n_classificado', regex=True, inplace=True)
df_pd_final_teste.replace({'CBO':'000000-n_classificado 000000-n_classificado'},'000000-n_classificado', regex=True, inplace=True)
df_pd_final_teste.replace({'CBO':'{ñ class}'}, '000000-n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.CBO

In [ ]:
#SUBSTITUINDO '{ñ class}' e '{ñ' por '000000-n_classificado', A INTENÇÃO DESTA SUBSTITUIÇÃO E INSERÇÃO DE UM CÓDIGO 'ZERADO' É PADRONIZAR A COLUNA JÁ QUE A COLUNA CID-10 POSSUI
#UMA ESPÉCIE DE CÓDIGO QUE O ANTECEDE, JÁ PLANEJANDO TAMBEM UMA POSSÍVEL SEPARAÇÃO DOS MESMOS, CRIANDO UMA NOVA COLUNA QUE RECEBERÁ O CÓDIGO.
df_pd_final_teste.replace({'CID-10':'{ñ class}'}, '000.0 n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.rename(columns={'CID-10':'cid_10'}, inplace=True)

In [ ]:
df_pd_final_teste.cid_10

In [ ]:
df_pd_final_teste.rename(columns={'CNAE2.0 Empregador.1':'cnae_empregador_1'}, inplace=True)
df_pd_final_teste.replace({'cnae_empregador_1':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.cnae_empregador_1

In [ ]:
df_pd_final_teste.rename(columns={'Munic Empr':'munic_empr'}, inplace=True)
df_pd_final_teste.replace({'Munic Empr':'{ñ class}'}, '000000-n_classificado', regex=True, inplace=True) 

In [ ]:
df_pd_final_teste.munic_empr

In [ ]:
df_pd_final_teste.rename(columns={'Natureza da Lesão':'natureza_lesao'}, inplace=True)
df_pd_final_teste.replace({'Natureza da Lesão':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.natureza_lesao

In [ ]:
df_pd_final_teste.rename(columns={'Origem de Cadastramento CAT':'orig_cad_cat'}, inplace=True)
df_pd_final_teste.replace({'orig_cad_cat':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)
df_pd_final_teste.replace({'orig_cad_cat':'{ñ class'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.orig_cad_cat

In [ ]:
df_pd_final_teste.rename(columns={'Parte Corpo Atingida':'parte_corpo_atingida'}, inplace=True)
df_pd_final_teste.replace({'parte_corpo_atingida':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.parte_corpo_atingida

In [ ]:
df_pd_final_teste.rename(columns={'UF Munic. Empregador':'uf_munic_empregador'}, inplace=True)
df_pd_final_teste.replace({'uf_munic_empregador':'Zerado'},0, regex=True, inplace=True)

In [ ]:
df_pd_final_teste.uf_munic_empregador

In [ ]:
df_pd_final_teste.rename(columns={'Emitente CAT':'emitente_cat'}, inplace=True)
df_pd_final_teste.replace({'emitente_cat':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.emitente_cat

In [ ]:
df_pd_final_teste.rename(columns={'Filiação Segurado':'filiacao_segurado'}, inplace=True)
df_pd_final_teste.replace({'filiacao_segurado':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.filiacao_segurado

In [ ]:
#INCONSISTÊNCIA ENCONTRADA '...' QUE SERÁ SUBSTITUÍDA POR UM VALOR QUE NÃO INTERFIRA NO PROCESSO DE CONVERSÃO PARA DATETIME
df_pd_final_teste.rename(columns={'Data Nascimento':'data_nascimento'}, inplace=True)
df_pd_final_teste.replace({'data_nascimento':r'^...$'}, value=0, regex=True, inplace=True)

In [ ]:
df_pd_final.tail(10)

In [ ]:
#AO CONSTATAR RETICÊNCIAS NA COLUNA CNPJ OPTAMOS POR PADRONIZAR COLOCANDO O NÚMERO 0
df_pd_final_teste.rename(columns={'CNPJ/CEI Empregador':'cnpj_empregador'}, inplace=True)
df_pd_final_teste.replace(to_replace={'cnpj_empregador':r'^...$'},value=0, regex=True, inplace=True)

In [ ]:
df_pd_final_teste.cnpj_empregador

In [ ]:
#INCONSISTÊNCIA ENCONTRADA '{ñ', QUE SERÁ SUBSTITUÍDA POR 'NaN', POIS SE TRATA DE UM DADO SENSÍVEL E AO ALTERAR PARA UM POSSÍVEL 'Não'
#ESTARIAMOS ADULTERANDO UM DADO QUANDO NÃO SABEMOS SE DE FATO NÃO HOUVE ÓBITO.
df_pd_final_teste.rename(columns={'Indica Óbito Acidente':'indica_obito_acidente'}, inplace=True)
df_pd_final_teste.replace({'indica_obito_acidente':'{ñ'}, np.NaN, regex=True, inplace=True)

In [ ]:
df_pd_final_teste.indica_obito_acidente

In [ ]:
df_pd_final_teste.rename(columns={'Munic Empr':'munic_empr'}, inplace=True)
df_pd_final_teste.replace({'munic_empr':'{ñ class}'}, 'n_classificado', regex=True, inplace=True)

In [ ]:
df_pd_final_teste.munic_empr

In [ ]:
df_pd_final_teste.rename(columns={'Espécie do benefício': 'especie_beneficio'}, inplace=True)

In [ ]:
df_pd_final_1 = df_pd_final_teste

In [ ]:
df_pd_final_1.head()

# INSIGHTS, FILTROS E GROUPBY

In [ ]:
pd.set_option('display.max_columns',170871)

In [ ]:
df_pd_final.head(50)

In [ ]:
ft_sex_masc = df_pd_final.Sexo = 'Masculino'
ft_sex_indeterminado = df_pd_final.Sexo == 'Indeterminado'
ft_sex_n_informado = df_pd_final.Sexo == 'Não Informado'
ft_sex_feminino = df_pd_final.Sexo == 'Feminino'

In [ ]:
#GROUPBY PARA VERIFICAÇÃO DOS MAIORES AGENTES CAUSADORES DE ACIDENTES
df_pd_final_1.groupby(['ag_causador_acidente']).size().sort_values(ascending=False)

In [ ]:
#VERIFICAÇÃO DOS MAIORES AGENTES CAUSADORES DE ACIDENTES ASSIM COMO SUA RESPECTIVA PLOTAGEM
df_pd_ag_causa = df_pd_final_1.ag_causador_acidente

In [ ]:
df_pd_final_1.groupby(['ag_causador_acidente']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='Agente Causador de Acidentes', ylabel = 'Frequencia', rot=90)

In [ ]:
df_pd_final_1.dtypes

In [ ]:
#VERIFICAÇÃO DAS LESÕES QUE MAIS OCORRERAM.
df_pd_final_1.groupby(['natureza_lesao']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='Natureza da Lesão', ylabel = 'Frequencia', rot=90)

In [ ]:
#VERIFICAÇÃO DAS MÉTRICAS ENTRE OCORRÊNCIAS DE ÓBITO COM AUXÍLIO DE PLOTAGEM
df_pd_final_1.groupby(['indica_obito_acidente']).size().sort_values(ascending=False)

In [ ]:
df_pd_final_1.groupby(['indica_obito_acidente'],dropna=False).size().sort_values(ascending=False).plot.pie(figsize=(8,8))

In [ ]:
#VERIFICADO QUE DAS 873 INDICAÇÕES DE ÓBITO SOMENTE 1 ESTÁ RECEBENDO - 'Pensão por Morte Acidente do Trabalho'
df_pd_final_1.groupby(['especie_beneficio']).size().sort_values(ascending=False)

In [ ]:
#VERIFICAÇÃO DAS ESPÉCIES DO BENEFÍCIO, OU SEJA IDENTIFICAR AS MÉTRICAS DE QUANTOS ESTÃO RECEBENDO ALGUM TIPO DE BENEFÍCIO 
#EM RELAÇÃO AOS QUE SE ENCONTRAM EM PROCESSO ADMINISTRATIVO.
df_pd_final_1.groupby(['especie_beneficio']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='Espécie do benefício', ylabel = 'Quantidade', rot=90)

In [ ]:
#VERIFICADO QUE A MAIORIA DOS ACIDENTES OCORRERAM COM INDIVÍDUOS DO SEXO MASCULINO
df_pd_final_1.groupby(['Sexo']).size().sort_values(ascending=False)

In [ ]:
#VERIFICADO QUE A MAIORIA DOS ACIDENTES SÃO DO TIPO TÍPICO
df_pd_final_1.groupby(['Tipo do Acidente']).size().head(10).sort_values(ascending=False).plot.bar(xlabel='Tipo do Acidente', ylabel = 'Frequencia', rot=90)

# PYSPARK
UTILIZAÇÃO DA BIBLIOTECA PYSPARK PARA DAR CONTINUIDADE AO TRATAMENTO DE DADOS.


In [ ]:
#CONFIGURAR A VARIÁVEL de AMBIENTE (SESSÃO) SPARK / VARIÁVEL DE SESSÃO QUE PERMITE O USO DO SPARK
spark = (
    SparkSession.builder
                .master('local')
                .appName('catweb')
                .getOrCreate()
)

In [ ]:
#MONTAGEM DO ESQUEMA DE COLUNAS CONTENDO A TIPAGEM DE CADA COLUNA VIA StructType
esquema = (
    StructType([
        StructField('agente_causador_acidente', StringType()),
        StructField('data_acidente', StringType()),
        StructField('cbo', StringType()),
        StructField('cid_10', StringType()),
        StructField('cnae2_empregador', IntegerType()),
        StructField('cnae2_empregador_1', StringType()),
        StructField('emitente_cat', StringType()),
        StructField('especie_beneficio', StringType()),
        StructField('filiacao_segurado', StringType()),
        StructField('indica_obito_acidente', StringType()),
        StructField('munic_empr', StringType()),
        StructField('natureza_da_lesao', StringType()),
        StructField('origem_cadastramento_cat', StringType()),
        StructField('parte_corpo_atingida', StringType()),
        StructField('sexo', StringType()),
        StructField('tipo_acidente', StringType()),
        StructField('uf_municipio_acidente', StringType()),
        StructField('uf_municipio_empregador', StringType()),
        StructField('data_acidente_1', StringType()),
        StructField('data_despacho_benefício', StringType()),
        StructField('data_acidente_2', StringType()),
        StructField('data_nascimento', StringType()),
        StructField('data_emissao_cat', StringType()),
        StructField('cnpj_cei_empregador', StringType())
    ])
)

In [ ]:
# CONVERSÃO DO DF, PANDAS PARA PYSPARK
df_spark = spark.createDataFrame(df_pd_final_1, schema=esquema)
df_spark.printSchema()
df_spark.show()

In [ ]:
#ANTES DE INICIAR O TRATAMENTO DE DADOS FAZ-SE NECESSÁRIO UM BACKUP, PARA EM CASO DE UM POSSÍVEL ERRO, OS DADOS ESTAREM ASSEGURADOS
df_spark_bckup = df_spark

# PRÉ ANÁLISE

In [ ]:
df_spark.printSchema()

In [ ]:
df_spark.show(truncate=False)

In [ ]:
df_spark.summary().show()

In [ ]:
df_spark.count()

In [ ]:
df_spark_distinct = df_spark.distinct()
print('Distinct ns.: '+str(df_spark_distinct.count()))

Distinct ns.: 156749


# TRATAMENTO DE DADOS

In [ ]:
#DROPAGEM DE LINHAS DUPLICADAS
#FOI VERIFICADO NO MERGE DO PANDAS QUE O NÚMERO DE LINHAS RESULTANTE É O MESMO DA DROPAGEM AQUI NO PYSPARK. 
#CONSTAMOS AQUI QUE FIZEMOS NO PADAS O COMANDO CONCAT DAS DFS PARA CONSTAR A INTEGRIDADE DA SOMATÓRIA DE LINHAS DOS DFS E OPTAMOS POR DROPA-LAS NO PYSPARK

In [ ]:
df_spark = df_spark.dropDuplicates()

In [ ]:
# VERIFICAÇÃO DE LINHAS
print('Contagem atual de rows: '+str(df_spark.count()))

In [ ]:
#DROPAGEM DAS COLUNAS DE DATAS REPETIDAS COM VALORES MENORES
#data_acidente e data_acidente_1 CONSTAM APENAS INFORMAÇÕES PARCIAIS COMO MES E ANO, 
#ENQUANTO A  coluna data_acidente_2 CONSTA DIA, MES e ANO REFERENTE A DATA ACIDENTE.

In [ ]:
df_spark = df_spark.drop(*['data_acidente','data_acidente_1'])

In [ ]:
df_spark.printSchema()

In [ ]:
# RENOMEAR A COLUNA data_acidente_2 PARA 'data_acidente', PERMANECENDO UMA SÓ COLUNA PARA DATA
df_spark = df_spark.withColumnRenamed('data_acidente_2', 'data_acidente')

In [ ]:
df_spark.printSchema()

In [ ]:
df_spark.summary().show()

In [ ]:
#DROPAGEM DA COLUNA uf_municipio_acidente
#COLUNA INCONGRUENTE COM AS DEMAIS INFORMAÇÕES GEOGRÁFICAS CONSTANTES NO DF
df_spark = df_spark.drop('uf_municipio_acidente')

In [ ]:
df_spark.printSchema()

In [ ]:
df_spark.summary().show()

#FILTROS/SPARK

In [ ]:
df_spark.printSchema()

In [ ]:
#FILTRO DA ÚNICA PESSOA QUE RECEBEU PENSÃO POR MORTE
df_spark.filter(df_spark.especie_beneficio == 'Pensão por Morte Acidente do Trabalho').show(truncate=False)

In [ ]:
df_spark.filter(df_spark.especie_beneficio == 'Pa').count()

116777

In [ ]:
#FILTRO DOS ACIDENTES QUE TIVERAM 'São Paulo' COMO UF DO EMPREGADOR
df_spark.filter(df_spark.uf_municipio_empregador == 'São Paulo').show(truncate=False)

In [ ]:
#FILTRO PARA RETORNO DA PROFISSÃO, SEXO, INDICAÇÃO DE ÓBITO E O TIPO DE ACIDENTE SOMENTE DAS PESSOAS QUE SOFRERAM ÓBITO
df_spark.select(F.col('cbo'), F.col('sexo'), F.col('indica_obito_acidente'), F.col('tipo_acidente')).filter(F.col('indica_obito_acidente') == 'Sim').show()

In [ ]:
#FILTRO PARA RETORNO DA PROFISSÃO, SEXO, NATUREZA DA LESAO E O TIPO DE ACIDENTE SOMENTE DAS PESSOAS DE PESSOAS QUE SOFRERAM POR CAUSA DE DOENÇA
df_spark.select(F.col('cbo'), F.col('sexo'), F.col('natureza_da_lesao'), F.col('tipo_acidente')).filter(F.col('tipo_acidente') == 'Doença').show()

In [ ]:
#FILTRO PARA RETORNO DA PROFISSÃO, SEXO, NATUREZA DA LESAO, INDICAÇÃO DE ÓBITO, TIPO DE ACIDENTE, DATA DE NASCIMENTO, PARTE DO CORPO ATINGIDA, DATA DO ACIDENTE DAS PESSOAS 
#QUE TIVERAM A CABEÇA COMO PARTE DO CORPO ATINGIDO
df_spark.select(F.col('cbo'), F.col('sexo'), F.col('natureza_da_lesao'), F.col('indica_obito_acidente'), F.col('tipo_acidente'), F.col('data_nascimento'), F.col('parte_corpo_atingida'), F.col('data_acidente')).filter(F.col('parte_corpo_atingida') == 'Cabeca, Nic').show(truncate=False)

In [ ]:
#FILTRO PARA RETORNO DA AGENTE CAUSADOR DO ACIDENTE, SEXO, NATUREZA DA LESAO, INDICAÇÃO DE ÓBITO E O TIPO DE ACIDENTE SOMENTE DAS PESSOAS DO SEXO FEMININO
df_spark.select(F.col('agente_causador_acidente'), F.col('sexo'), F.col('natureza_da_lesao'), F.col('indica_obito_acidente'), F.col('tipo_acidente')).filter(F.col('sexo') == 'Feminino').show(truncate=False)

In [ ]:
#FILTRO PARA RETORNO DA AGENTE CAUSADOR DO ACIDENTE, CBO, SEXO, NATUREZA DA LESAO, TIPO DO ACIDENTE SOMENTE QUANDO O AGENTE CAUSADOR FOR BICICLETA
df_spark.select(F.col('agente_causador_acidente'), F.col('cbo'), F.col('sexo'), F.col('natureza_da_lesao'), F.col('tipo_acidente')).filter(F.col('agente_causador_acidente') == 'Bicicleta').show()


In [ ]:
# Filtro para retorno do sexo e incidencia óbito
df_spark.select(F.col('sexo'), F.col('indica_obito_acidente'), F.col('sexo') == 'Feminino').filter(F.col('indica_obito_acidente') == 'Sim').show()

# WINDOW FUNCTIONS


In [ ]:
w0 = Window.orderBy('data_emissao_cat').partitionBy('indica_obito_acidente')
w1 = Window.orderBy('uf_municipio_empregador')
w2 = Window.orderBy('codigo_municipio')
w4 = Window.partitionBy('data_emissao_cat').orderBy('especie_beneficio')
w5 = Window.partitionBy('cid_10').orderBy('especie_beneficio')

In [ ]:
df_spark.withColumn('dense_rank', dense_rank().over(w1)).show()

In [ ]:
df_spark = df_spark.withColumn('dense_rank', dense_rank().over(w5))

In [ ]:
df_spark = df_spark.withColumn('rank', rank().over(w4))

In [ ]:
df_spark.show()

# SPARKSQL

In [ ]:
#CRIAÇÃO DE TABELA EM SQL PARA EXECUTAR ALGUMAS QUERYS, UTILIZANDO A LINGUAGEM SQL.

In [ ]:
spark.sql('show databases').show()

In [ ]:
spark.sql('create database df_spark').show()

In [ ]:
spark.sql('show databases').show()

In [ ]:
spark.sql('use df_spark')

In [ ]:
spark.sql('SHOW TABLES').show()

In [ ]:
df_spark.write.saveAsTable('df_spark')

In [ ]:
spark.sql('show tables').show()

In [1]:
spark.sql('select * from df_spark').show()

NameError: ignored

# INSIGHTS SPARKSQL

In [ ]:
#INSIGHT 1 - ORDENAÇÃO DA NATUREZA DA LESÃO POR ORDEM ALFABÉTICA
spark.sql('select natureza_da_lesao FROM df_spark ORDER BY natureza_da_lesao asc;').show(truncate=False)

In [ ]:
#INSIGHT 2 - EXIBIÇÃO DE TODAS AS OCORRÊNCIAS DE ÓBITOS
spark.sql('select * FROM df_spark WHERE indica_obito_acidente = "Sim";').show(truncate=False)

In [ ]:
#INSIGHT 3 - SELEÇÃO DE TODOS OS ACIDENTES QUE 'Animal Vivo' como AGENTE CAUSADOR DO ACIDENTE
spark.sql('select * FROM df_spark WHERE agente_causador_acidente = "Animal Vivo";').show(truncate=False)

In [ ]:
#INSIGHT 4 - SELEÇÃO DOS ACIDENTES QUE TIVERAM O DEDO COMO PARTE DO CORPO ATINGIDA
spark.sql('select * FROM df_spark WHERE parte_corpo_atingida = "Dedo";').show(truncate=False)

In [ ]:
#INSIGHT 5 - SELECÃO DOS ACIDENTES QUE OCORRERAM COM PESSOAS QUE SÃO ENFERMEIRO(A)
spark.sql('select * FROM df_spark WHERE cbo = "223505-Enfermeiro";').show(truncate=False)

In [ ]:
#INSIGHT 6 - ORDENAÇÃO DOS MUNICÍPIOS EM ORDEM ALFABÉTICA.
spark.sql('select uf_municipio_empregador FROM df_spark ORDER BY uf_municipio_empregador asc;').show(truncate=False)

# CONVERSÃO PARA PANDAS 

In [ ]:
#OPTAMOS POR CONVERTER PARA O PANDAS PARA FACILITAR O ENVIO DOS ARQUIVOS PARA OS BANCOS.

In [ ]:
df_pandas = df_spark.toPandas()

In [ ]:
df_catweb_tratado = df_pandas.copy

#INSTALAÇÃO DAS BIBLIOTECAS DO CONECTOR MYSQL

In [ ]:
!pip install mysql-connector-python

In [ ]:
!pip install pymysql

In [ ]:
import mysql.connector
import pymysql
from sqlalchemy import create_engine

In [ ]:
import google.auth
from google.auth.transport.requests import Request
import sqlalchemy


#ENVIO PARA O BUCKET NA GCP

In [ ]:
df_pandas.to_csv()

In [ ]:
#IMPORTAÇÃO DO DF TRATADO PARA O BUCKET
df_pandas.to_csv('gs://bc26-datasets-ivana/catweb_tratado_export.csv',storage_options={'token':'token_gcp'})

#ENVIO PARA MYSQL

In [ ]:
conexao = 'ip_address'
nome_do_banco = 'database_name'
usuario = 'user'
senha = 'password' 
engine = create_engine( "mysql+pymysql://{user}:{pw}@{host}/{db}"
              .format(host=conexao, db=nome_do_banco, user= usuario, pw=senha))

df_pandas.to_sql("df_pandas",engine,index=True,index_label='id')

#ENVIO PARA O MONGO

In [ ]:
#CONECTOR DO MONGO ATLAS
uri = "url_mongo_db"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile='certificate_key')

In [ ]:
#ESCOLHA DO DATABASE E COLEÇÃO - ORIG
db = client['catweb_tratado'] #DATABASE
colecao_catweb = db['catweb_db_tratado'] #COLEÇÃO
db_tratado = client['catweb_db_tratado']
colecao_tratado = db['catweb_tratado']

In [ ]:
df_catweb_tratado = df_pandas.to_dict("records")
colecao_tratado.insert_many(df_catweb_tratado)

In [ ]:
colecao_tratado.count_documents({})

#FIM DO PROCESSO